In [2]:
# Carregar llibreries

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np
import random
import time
from google.colab import files

In [3]:
# Carrega el fitxer Excel

# INFORME GPG: Inventari de drets immaterials - Dates de vigència

# nom_fitxer = '/content/Inventaridedretsimmaterials-Datesdevigència (2).xlsx'  # Canvia a la ruta correcta del teu fitxer
# df = pd.read_excel(nom_fitxer, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Carrega l'arxiu XLSX
uploaded = files.upload()

# Obtenir el nom de l'arxiu carregat
nom_arxiu = list(uploaded.keys())[0]

# Llegir l'arxiu XLSX amb pandas
df = pd.read_excel(nom_arxiu, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Selecciona les columnes que necessites
columnes_seleccionades = [
    "Codi registre",
    "Registre d'inscripció",
    "Data fi vigència",
    "Data inici vigència",
    "Data renovació",
    "Catàleg",
    "Estat",
    "Titular",
    "Número d'inventari"
]

df = df[columnes_seleccionades]


Saving Inventaridedretsimmaterials-Datesdevigència (2).xlsx to Inventaridedretsimmaterials-Datesdevigència (2).xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# Filtrar XLSX

# Obtenir la data d'avui
avui = datetime.now()

df2 = df[
    (df['Estat'] == 'Alta') &
    (df["Catàleg"] == 'Marca') &
    (df["Registre d'inscripció"] == "Oficina Espanyola de Patents i Marques (OEPM)") &
    (df["Data fi vigència"] < avui)
       ]
# print(df2["Codi registre"])

In [5]:
def separar(cadena):
    # Patró de cerca per la data (dd/mm/yyyy)
    patro_data = re.compile(r'\b(\d{1,2}/\d{1,2}/\d{4})\b')

    # Cerca la data a la cadena
    coincidencia_data = patro_data.search(cadena)

    if coincidencia_data:
        # Si es troba la data, separa el text i la data
        text = cadena[:coincidencia_data.start()].strip()
        data = coincidencia_data.group()
    else:
        # Si no es troba la data, retorna la cadena completa com a text i deixa la data buida
        text = cadena
        data = ""

    return text, data

# Exemple d'ús
# cadena = "PUBL.CONCES.RENOV. DE F.RESOL. 06/11/2023"
# text, data = separar(cadena)


In [46]:
def dades(codi):

    url = "http://consultas2.oepm.es/ceo/jsp/busqueda/consultaExterna.xhtml?numExp=M" + codi

    dades = {}  # Crear un diccionari per emmagatzemar les dades

    dades['Codi'] = codi

    try:
        response = requests.get(url)

        if response.status_code == 200:
            # Parseja el contingut HTML amb BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Troba l'element <h4> amb el text "Actos de tramitación:"
            h4_actos = soup.find('h4', string=lambda text: "Actos de tramitación:" in text if text else False)
            h4_actos = h4_actos.find_next('div', class_='datos')

            # Troba tots els elements <td> amb l'atribut role="gridcell"
            td_elements = h4_actos.find_all('td', {'role': 'gridcell'})

            # Verifica si hi ha algun element
            if td_elements:
                # Obtenir l'últim element i extreure'n el text
                dades['last_td'] = td_elements[-1].get_text(strip=True)
                dades['Data publicació'] = td_elements[-2].get_text(strip=True)

                dades['Anotació'], dades['Data anotació'] = separar(dades['last_td'])

            # Busca l'element <h4> que contingui el text "Estado:"
            estado_h4 = soup.find('h4', string=lambda text: "Estado:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if estado_h4:
                dades['Estat'] = estado_h4.find_next('p').get_text(strip=True)

            # Troba l'element <div> amb l'id "marco-imagen-sitamar"
            div_marco_imagen = soup.find('div', {'id': 'marco-imagen-sitamar'})

            if div_marco_imagen:
                # Troba l'element <img> dins del div
                img_element = div_marco_imagen.find('img')

                if img_element:
                    # Obté el valor de l'atribut src
                    src_valor = img_element.get('src', '')

                    # Imprimeix el resultat
                    dades['Imatge'] = "http://consultas2.oepm.es"+src_valor
                else:
                    dades['Imatge'] = ""

            # Troba l'element <h3>
            h3_element = soup.find('h3')

            if h3_element:
                # Obté el text de l'element <h3>
                text_h3 = h3_element.get_text(strip=True)

                # Utilitza una expressió regular per seleccionar el segon grup
                match = re.match(r'(.*)\s-\s(.*)', text_h3)

                if match:
                    segon_grup = match.group(2)
                    dades['Denominació'] = segon_grup

            # Troba l'element <h4>
            h4_element = soup.find('h4', string=lambda text: "Tipo:" in text if text else False)

            if h4_element:
                # Obtenir el text de l'element <h4>
                text_h4 = h4_element.find_next('p').get_text(strip=True)

                dades['Tipus'] = text_h4

            # Extreu el text que segueix a <span class="campo">Nombre:</span>
            nom_span = soup.find('span', class_='campo', string="Nombre:")
            if nom_span:
                text_nom = nom_span.find_next('div', class_='datos-sitamar-divSubRight').text.strip()
                # Comprova si hi ha l'etiqueta <br/>
                if "\n" in text_nom:
                    # Divideix el text en dos blocs amb l'etiqueta <br />
                    motiu, titular = map(str.strip, text_nom.split('\n', 1))
                    dades['Titular OEPM'] = titular
                    dades['Motiu'] = motiu
                else:
                    # Si no hi ha <br />, posa tot el text a 'titular'
                    dades['Titular OEPM'] = text_nom
                    dades['Motiu'] = ""

        else:
            print(f"S'ha produït un error en la sol·licitud. Codi d'estat: {response.status_code}")

    except requests.RequestException as e:
        print(f"Error de connexió: {e}")

    dades['Url'] = url
    dades['Titular GPG'] = df2['Titular'][df2['Codi registre'] == codi].values[0]
    dades['Inici vigència GPG'] = pd.to_datetime(df2["Data inici vigència"][df2['Codi registre'] == codi]).dt.date.values[0]
    dades['Fi vigència GPG'] = pd.to_datetime(df2["Data fi vigència"][df2['Codi registre'] == codi]).dt.date.values[0]
    dades['Núm. secció'] = df2["Número d'inventari"][df2['Codi registre'] == codi].values[0]

    return dades  # Retorna el diccionari amb les dades


In [7]:
'''
# Exemple d'ús

resultats = dades("2551345")

# Ara pots fer el que vulguis amb les dades obtingudes
if resultats:
    # Fer quelcom amb les dades, com ara imprimir-les
    print("Dades obtingudes:")
    print(f"codi: {resultats.get('Codi')}")
    print(f"denominació: {resultats.get('Denominació')}")
    print(f"estado_text: {resultats.get('Estat')}")
    print(f"data_publicacio: {resultats.get('Data publicació')}")
    print(f"anotacio: {resultats.get('Anotació')}")
    print(f"anotacio_data: {resultats.get('Data anotació')}")
    print(f"imatge: {resultats.get('Imatge')}")
    print(f"titular: {resultats.get('Titular OEPM')}")
    print(f"motiu: {resultats.get('Motiu')}")
    print(f"tipus: {resultats.get('Tipus')}")
    print(f"Url: {resultats.get('Url')}")
'''

'\n# Exemple d\'ús\n\nresultats = dades("2551345")\n\n# Ara pots fer el que vulguis amb les dades obtingudes\nif resultats:\n    # Fer quelcom amb les dades, com ara imprimir-les\n    print("Dades obtingudes:")\n    print(f"codi: {resultats.get(\'Codi\')}")\n    print(f"denominació: {resultats.get(\'Denominació\')}")\n    print(f"estado_text: {resultats.get(\'Estat\')}")\n    print(f"data_publicacio: {resultats.get(\'Data publicació\')}")\n    print(f"anotacio: {resultats.get(\'Anotació\')}")\n    print(f"anotacio_data: {resultats.get(\'Data anotació\')}")\n    print(f"imatge: {resultats.get(\'Imatge\')}")\n    print(f"titular: {resultats.get(\'Titular OEPM\')}")\n    print(f"motiu: {resultats.get(\'Motiu\')}")\n    print(f"tipus: {resultats.get(\'Tipus\')}")\n    print(f"Url: {resultats.get(\'Url\')}")\n'

In [42]:
# Baixar la informació

marques_revisades = pd.DataFrame()

id = df2["Codi registre"]

print("Marques a processar: ", len(id), "\n")

for index, x in enumerate(id):
    print(str(round(index/len(id)*100, 2))+"%")
    resultats = dades(x)
    marques_revisades = pd.concat([marques_revisades, pd.DataFrame([resultats])], ignore_index=True)
    time.sleep(random.uniform(10, 15))
print("100.00 %")

marques_revisades.drop('last_td', axis=1, inplace=True)
marques_revisades

Marques a processar:  28 

0.0%
100.00 %


,Codi,Data publicació,Anotació,Data anotació,Estat,Imatge,Denominació,Tipus,Titular OEPM,Motiu,Url,Titular GPG,Inici vigència GPG,Fi vigència GPG,Núm. secció
0,2550771,08/09/2023,ANOTACION SOLICITUD RENOVACION,,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,CATALONIA AIRPORTS,Mixta,GENERALITAT DE CATALUNYA. DEPARTAMENT DE POLIT...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Territori,2003-07-04,2023-07-04,4763


In [43]:
# Defineix l'ordre desitjat de les columnes
ordre_columnes = ['Codi',	'Núm. secció', 'Denominació',	'Tipus',	'Estat',	'Data publicació',	'Anotació',	'Data anotació',	'Titular OEPM',	'Motiu', 'Imatge',	'Url', 'Titular GPG', 'Inici vigència GPG', 'Fi vigència GPG']

# Reordena les columnes del DataFrame
marques_revisades = marques_revisades[ordre_columnes]
marques_revisades

,Codi,Núm. secció,Denominació,Tipus,Estat,Data publicació,Anotació,Data anotació,Titular OEPM,Motiu,Imatge,Url,Titular GPG,Inici vigència GPG,Fi vigència GPG
0,2550771,4763,CATALONIA AIRPORTS,Mixta,En vigor,08/09/2023,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA. DEPARTAMENT DE POLIT...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Territori,2003-07-04,2023-07-04


In [44]:
marques_revisades.to_excel(f'marques_oepm_revisades_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx', index=False)

In [45]:
files.download(f'marques_oepm_revisades_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>